## Информация о каком количестве генов есть в исследовании? Есть ли в данных дубликаты по генам? Что еще вы можете сказать о данных?

Для каждого из 18 образцов есть данные по 22514 уникальным наименованиям. Сырые данные содержали 45018 значений интенсивности пикселей. Из них были отфильтрованы 23930 признаков, так чтобы 15 из 18 образцов или все 6 образцов из одной категории удовлетворяли функции фильтрации. Функция фильтрации проверяла, чтобы для извлеченного признака из образца поле ControlType равнялось 0, поле gIsFound равнялось 1, поле gIsPosAndSignif равнялось 1 (позитивный и значимый сигнал), поле gIsWellAboveBG равнялось 1 (сигнал значительно выше шума). Полученная таблица значений была нормализована по квантилям. Для дублирующихся признаков сигнал был усреднен. В результате было получено 22514 уникальных признаков. Для каждого образца каждому признаку соответствует прологарифмированное по модулю 2 значение сигнала, полученного в результате эксперимента.

In [1]:
# Load datasets and print statistics
import pandas as pd

data = pd.read_csv("data.txt", sep="\t", index_col=0).T
values = pd.DataFrame(["control"] * 6 + ["low_dose"] * 6 + ["high_dose"] * 6, index = data.index, columns = ["category"])

print("Number of samples:", len(data.index))
print("Number of features:", len(data.columns))
print("Number of unique features:", len(data.columns.unique()))


Number of samples: 18
Number of features: 22514
Number of unique features: 22514


In [2]:
# Calculate mean expression values per feature by category
categories = ["control", "low_dose", "high_dose"]
mean_by_category = pd.DataFrame(index = categories, columns=data.columns)


for i in categories:
  samples = values[values["category"]==i].index.values
  mean_by_category.loc[i] = data.loc[samples].mean()
  
print(mean_by_category)

ID_REF    A_51_P100034 A_51_P100063 A_51_P100099 A_51_P100155 A_51_P100174  \
control         12.365        6.108        7.525      10.6935        6.219   
low_dose       12.1485      6.13333      7.50967      10.5788      6.07767   
high_dose      12.1823        6.648      7.48667      10.4737       6.1995   

ID_REF    A_51_P100181 A_51_P100227 A_51_P100246 A_51_P100289 A_51_P100327  \
control        6.72883      8.21417      9.63917      8.42467      7.32417   
low_dose       6.82517      8.24383      9.37933       8.2955        7.315   
high_dose      6.73717      8.28217      9.32567      8.35183      7.60833   

ID_REF     ... A_52_P995381 A_52_P996032 A_52_P996473 A_52_P99665 A_52_P99670  \
control    ...      7.34317      8.56733      6.01233     8.04533      7.6015   
low_dose   ...        7.123      8.09767      6.09433     8.00433     7.32817   
high_dose  ...      7.04883      8.01417      6.09583     8.24167     7.21467   

ID_REF    A_52_P997209 A_52_P997449 A_52_P99807 A

In [3]:
from sklearn.tree import DecisionTreeClassifier

control_low_samples = values[(values["category"]=="control") | (values["category"]=="low_dose")].index.values
control_low_data = data.loc[control_low_samples]
control_low_values = values.loc[control_low_samples]

low_features = set()
for i in range(10000):
  clf_low = DecisionTreeClassifier(max_depth=2)
  clf_low.fit(control_low_data.values, control_low_values.values)

  for i, v in enumerate(clf_low.feature_importances_):
    if v != 0:
      low_features.add(control_low_data.columns[i])


control_high_samples = values[(values["category"]=="control") | (values["category"]=="high_dose")].index.values
control_high_data = data.loc[control_high_samples]
control_high_values = values.loc[control_high_samples]

high_features = set()
for i in range(10000):
  clf_high = DecisionTreeClassifier(max_depth=2)
  clf_high.fit(control_high_data.values, control_high_values.values)

  for i, v in enumerate(clf_high.feature_importances_):
    if v != 0:
      high_features.add(control_high_data.columns[i])

print(len(low_features), "control vs low_dose features:", low_features)
print()
print(len(high_features), "control vs high_dose features:", high_features)
print()

common_features = low_features & high_features
print(len(common_features), "common for both two-class problems features:", common_features)
print()

270 control vs low_dose features: {'A_52_P1013551', 'A_52_P244736', 'A_51_P345470', 'A_51_P382618', 'A_51_P359374', 'A_52_P303041', 'A_51_P370099', 'A_51_P416059', 'A_52_P470316', 'A_51_P429366', 'A_51_P446012', 'A_52_P289274', 'A_52_P556328', 'A_51_P254583', 'A_51_P200477', 'A_51_P175354', 'A_52_P166770', 'A_52_P647906', 'A_52_P446011', 'A_51_P256384', 'A_52_P107354', 'A_52_P651078', 'A_51_P212854', 'A_52_P539361', 'A_52_P318040', 'A_52_P121916', 'A_51_P223199', 'A_51_P406145', 'A_52_P621603', 'A_51_P372473', 'A_51_P453043', 'A_52_P466853', 'A_51_P145402', 'A_52_P225249', 'A_51_P382310', 'A_51_P518001', 'A_52_P608024', 'A_52_P162957', 'A_52_P385824', 'A_51_P416689', 'A_52_P656659', 'A_52_P167166', 'A_51_P400816', 'A_51_P504624', 'A_51_P229076', 'A_51_P431919', 'A_52_P480709', 'A_51_P324871', 'A_51_P496432', 'A_51_P401193', 'A_51_P251069', 'A_51_P188126', 'A_52_P331215', 'A_51_P357207', 'A_51_P451574', 'A_51_P439426', 'A_52_P512301', 'A_52_P475870', 'A_52_P99470', 'A_52_P35174', 'A_52_